In [89]:
import pandas as pd
import urllib.parse
import sqlalchemy
from sqlalchemy import create_engine, text
import openpyxl
import re
from datetime import datetime
import pytz
import os
import polars as pl
from lib.transform import CreateTableInSQLServer, ARBalanceListing
SQL_SERVER = 'TOM'
DB_Name = 'SSRPA_310492'
User_Name = 'TOM'
Password = 'Silversea'
class StaffList:
    
    def __init__(self, file_path):
        self.file_path = file_path

    def process_files(self):
        # Process each Staff
        file_path = self.file_path
        wb = openpyxl.load_workbook(file_path)
        sheet = wb.worksheets[1]
        last_row = sheet.max_row
        D = {}

        # Find last row with "Pay Type:"
        for row in range(last_row - 6, last_row + 1):
            cell = sheet[f"C{row}"]
            if cell.value == "Pay Type:":
                last_row = row + 1
                break

        # Find first row with "StaffID"
        for row in range(1, last_row):
            if sheet[f"C{row}"].value is not None:
                if sheet[f"C{row}"].value == "Staff ID":
                    first_row = row + 1  # First position of StaffID
                    break
        # Find first row with "Full Name:"
        first_range = 0
        last_range = 0
        for row in range(1, last_row):
            if sheet[f"C{row}"].value is not None:
                if sheet[f"C{row}"].value == "Full Name:":
                    if first_range == 0 and last_range == 0:
                        first_range = row
                    elif first_range != 0 and last_range == 0:
                        last_range = row
                    else:
                        size_range = last_range - first_range
                        break
        # Process the data
        for i in range(1, int((last_row - first_row + 1) / size_range + 1)):
            StaffID = sheet[f"C{(i-1)*size_range + first_row + 1}"].value
            ReportName = sheet[f"D{(i-1)*size_range + first_row + 1}"].value
            StaffNameNull = sheet[f"D{(i-1)*size_range + first_row + 2}"].value
            StaffOffice = sheet[f"D{(i-1)*size_range + first_row + 4}"].value
            StaffBU = sheet[f"D{(i-1)*size_range + first_row + 5}"].value
            StaffDepartment = sheet[f"D{(i-1)*size_range + first_row + 6}"].value
            ReportingManager = sheet[f"K{(i-1)*size_range + first_row + 5}"].value
            StaffStatus = sheet[f"O{(i-1)*size_range + first_row + 1}"].value
            D[i] = [
                StaffID,
                ReportName,
                StaffNameNull,
                StaffOffice,
                StaffBU,
                StaffDepartment,
                ReportingManager,
                StaffStatus,
            ]

        # Create DataFrame
        df = pd.DataFrame.from_dict(
            D,
            orient="index",
            columns=[
                "StaffID",
                "ReportName",
                "StaffNameNull",
                "StaffOffice",
                "StaffBU",
                "StaffDepartment",
                "ReportingManager",
                "StaffStatus",
            ],
        )
        return df

result_df = StaffList(r"C:\Users\tomda\OneDrive\Desktop\Python\Source ACS\WIPAR_Aging").process_files()
result_df.head()

SSRPA WIP AR Aging 102022.xlsx
SSRPA WIP AR Aging 102023.xlsx
SSRPA WIP AR Aging 102024.xlsx
SSRPA WIP AR Aging 112022.xlsx
SSRPA WIP AR Aging 112023.xlsx
SSRPA WIP AR Aging 112024.xlsx
SSRPA WIP AR Aging 12022.xlsx
SSRPA WIP AR Aging 12023.xlsx
SSRPA WIP AR Aging 12024.xlsx
SSRPA WIP AR Aging 122022.xlsx
SSRPA WIP AR Aging 122023.xlsx
SSRPA WIP AR Aging 122024.xlsx
SSRPA WIP AR Aging 22022.xlsx
SSRPA WIP AR Aging 22023.xlsx
SSRPA WIP AR Aging 22024.xlsx
SSRPA WIP AR Aging 32022.xlsx
SSRPA WIP AR Aging 32023.xlsx
SSRPA WIP AR Aging 32024.xlsx
SSRPA WIP AR Aging 42022.xlsx
SSRPA WIP AR Aging 42023.xlsx
SSRPA WIP AR Aging 42024.xlsx
SSRPA WIP AR Aging 52022.xlsx
SSRPA WIP AR Aging 52023.xlsx
SSRPA WIP AR Aging 52024.xlsx
SSRPA WIP AR Aging 62022.xlsx
SSRPA WIP AR Aging 62023.xlsx
SSRPA WIP AR Aging 62024.xlsx
SSRPA WIP AR Aging 72022.xlsx
SSRPA WIP AR Aging 72023.xlsx
SSRPA WIP AR Aging 72024.xlsx
SSRPA WIP AR Aging 82022.xlsx
SSRPA WIP AR Aging 82023.xlsx
SSRPA WIP AR Aging 82024.xlsx
S

,ClientIdSubId,LastPaymentDate,ARTotal,WIPTotal,AR0030,WIP0030,AR3160,WIP3160,AR6190,WIP6190,AR91120,WIP91120,AR121150,WIP121150,AR151180,WIP151180,AROver180,WIPOver180,LastPaymentAmount,CutOff
0,000000146,2024-04-09,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1350.00,2022-10-31
1,002280013,2017-01-04,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,300.00,2022-10-31
2,003342423,2017-01-04,0.0,437.50,0.0,400.0,0.0,37.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,300.00,2022-10-31
3,007445733,2024-07-22,0.0,11254.55,0.0,-412.5,0.0,150.0,0.0,1012.5,0.0,975.0,0.0,825.0,0.0,225.0,0.0,8479.55,2000.00,2022-10-31
4,010548832,2024-04-15,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,980.00,2022-10-31


In [67]:
def find_date( txt):
    dates = re.findall(r"\d{1,2}/\d{1,2}/\d{4}", txt)
    first_date = dates[1] if dates else None
    return first_date

def clean_column_name( col_name):
    return re.sub(r"[^a-zA-Z0-9]", "", col_name)
temp = pl.read_excel(r"C:\Users\tomda\OneDrive\Desktop\Python\Source Spiegel\WIP_Activity\SSRPA WIP Activity Net of Retainers 12022.xlsx", sheet_id=2)
df = temp.to_pandas()
df = df.astype(str)

# Find row contains header
found = False
for row in df.index:
    for col_idx in range(df.shape[1]):
        if df.iloc[row, col_idx]:
            if df.iloc[row, col_idx] == "WIP Beg Balance":
                row_header = row
                found = True
                break
    if found:
        break
# List all cols name to keep
# Keep ClientIdSubId column
keep_cols = []
found = False
for row in range(row_header, df.shape[0]):
    for col_idx in range(df.shape[1]):
        if df.iloc[row, col_idx]:
            if df.iloc[row, col_idx][:18] == "Client ID Sub ID :":
                keep_cols.append(col_idx)
                found = True
                break
    if found:
        break
for col_idx in range(df.shape[1]):
    if (df.iloc[row_header, col_idx] != "None") and (df.iloc[row_header, col_idx] != "") :
        keep_cols.append(col_idx)
# Determine CutOffDate
found = False
for row in range(16):  # Because this cell always before 16 rows
    for col in df.columns:
        if df.at[row, col] and isinstance(df.at[row, col], str):
            if df.at[row, col][:3] == "PTD":

                row_cutoff, col_cutoff = row, col
                found = True
                break  # Break the inner loop
    if found:
        break
CutOffDate = find_date(df.at[row_cutoff, col_cutoff])
df = df.astype(str)
df = df.iloc[:, keep_cols]
df.columns = [
    clean_column_name(df.iloc[row_header, col])
    for col in range(df.shape[1])
            ]
df.columns.values[0] = "Type"
df["ClientIdSubId"] = df["Type"]
df = df[(df["Type"] != "None") & (df["Type"] != "RTD") & (df["Type"] != "")].reset_index(
    drop=True
)
for row in reversed(df.index):
    if df.iloc[row, 0] == "Grand Totals":
        df.drop(df.index[row:], inplace=True)
        break
df["ClientIdSubId"] = df["ClientIdSubId"].replace('PTD', None)
print(row_header)
df.head()

2


,Type,WIPBegBalance,ProductionAmounts,WIPBalance,BilledAmounts,WriteUpDown,ProgressBilled,ProgressApplied,WIPEndBalance,WIP,RelievedWIPAdjust,ClientIdSubId
0,Client ID Sub ID : 2055.001,None,None,None,None,None,None,None,None,None,None,Client ID Sub ID : 2055.001
1,PTD,0,13.5,13.5,0,0,0,0,13.5,0.00%,0.00%,None
2,Client ID Sub ID : 2092.001,None,None,None,None,None,None,None,None,None,None,Client ID Sub ID : 2092.001
3,PTD,0,0,0,0,0,0,0,0,0.00%,0.00%,None
4,Client ID Sub ID : 2097.001,None,None,None,None,None,None,None,None,None,None,Client ID Sub ID : 2097.001
